# Statistical Analysis of the Execution-Based Results

## Extract error mitigation flow data from testresults

In [ ]:
import os
import pandas as pd
import json

from codetransbench.utils.dataset_information import language_pairs
from codetransbench.utils.config import load_config

from codetransbench.translation.translate_open_source import FILE_EXTENSIONS


In [ ]:
path_to_config = "../codetransbenchmark/config/config.yaml"
if path_to_config:
    config = load_config(path_to_config)
else:
    config = load_config()
output_dir = config.output_dir
dataset_dir = config.dataset_dir
testresults_dir = config.testresults_dir

### Set output post-processing
Set to None to use the default

In [ ]:
opp = None#"controlled"

### Extract the data

In [ ]:
import build_combined_reporting as build_combined_reporting
build_combined_reporting.main_general(config, opp)

# Work with loaded data for analytics

In [ ]:
import pandas as pd
from pathlib import Path


if opp:
    combined_raw_count = pd.read_csv(Path("./data") / f"combined_exec_{opp}.csv")
    raw_combined = pd.read_csv(Path("./data") / f"raw_combined_mitigation_{opp}.csv", index_col=0)
    raw_combined

In [ ]:
if opp == None:
    combined_raw_count = pd.read_csv(Path("./data") / "combined_exec.csv")
    raw_combined = pd.read_csv(Path("./data") / "raw_combined_mitigation.csv", index_col=0)
    raw_combined
    combined_raw_count


## Cleanup of identifyers

In [ ]:
# Prettify identifiers
raw_combined["model_name"] = raw_combined["model_name"].str.split(r'_').str.get(1)

In [ ]:
result_cat = ["success", "compile", "runtime", "incorrect"]

# Filtering of the tasks with faulty source files

In [ ]:
combined_without_faluty_source = raw_combined.query("result_1 != 'faulty source' and result_1 != 'unicode error'")
combined = combined_without_faluty_source[['dataset', 'source_lang', 'target_lang', 'model_name', 'template', 'result_1', 'target_filename']].groupby(['dataset', 'model_name', 'template', 'source_lang', 'target_lang', 'result_1']).count()
combined_without_faluty_source.loc[:,"result_1"] = pd.Categorical(combined_without_faluty_source.loc[:,"result_1"], categories = result_cat, ordered=True)
if opp == None:
    combined_without_faluty_source.loc[:,"result_2"] = pd.Categorical(combined_without_faluty_source.loc[:,"result_2"], categories = result_cat, ordered=True)
    combined_without_faluty_source.loc[:,"result_3"] = pd.Categorical(combined_without_faluty_source.loc[:,"result_3"], categories = result_cat, ordered=True)
combined_without_faluty_source

### Setup Matplotlib plotting

In [ ]:
pd.options.plotting.backend = "matplotlib"
import matplotlib.pyplot as plt
plt.style.use("thesis_style_sheet.mplstyle")

### Overlap in outcome in the 1st iteration

In [ ]:
combined_without_faluty_source.filter(['dataset', 'source_lang', 'target_lang', 'model_name', 'template', 'result_1', 'source_filename', 'target_filename']).groupby(["source_filename", "target_filename", "result_1"]).count()

In [ ]:
combined_without_faluty_source.loc[combined_without_faluty_source.model_name.str.contains("mistral")].filter(['model_name', 'template', 'result_1', 'source_filename', 'target_filename']).groupby(["source_filename", "target_filename", "result_1"]).count()

In [ ]:
combined_without_faluty_source.loc[combined_without_faluty_source.model_name == "mistral"].filter(['model_name', 'template', 'result_1', 'source_filename', 'target_filename']).groupby(["source_filename", "target_filename", "result_1"]).count()

In [ ]:
combined_without_faluty_source.loc[combined_without_faluty_source.template == "controlled_md"].filter(['model_name', 'template', 'result_1', 'source_filename', 'target_filename']).groupby(["source_filename", "target_filename", "result_1"]).count()

Distribution of successful translation pairs vs the number of models that were successful. Template: controlled_md
If a file was translated successfully, how many models were successful.

DO NOT USE THIS

In [ ]:
# success_dist_files = combined_without_faluty_source.query("template == 'controlled_md' and result_1 == 'success'").filter(['model_name', 'template', 'result_1', 'source_filename', 'target_filename']).groupby(["source_filename", "target_filename", "result_1"]).count().rename(columns={"model_name": "successful models"}).groupby("successful models").count().rename(columns={"template":"Ratio of tasks"})
# success_dist_files.div(success_dist_files.sum()).plot(kind="bar", rot=0)

# Results without iteration

In [ ]:
combined_reset = combined.reset_index()
combined_reset.loc[:,"result_1"] = pd.Categorical(combined_reset.loc[:,"result_1"], categories = result_cat, ordered=True)

In [ ]:

combined_pivot = pd.pivot_table(combined_reset, values="target_filename", index=['dataset', 'model_name', 'template', 'source_lang', 'target_lang'], columns=["result_1"], aggfunc=sum,
               fill_value=0,
               margins=True)
#combined_pivot = combined_pivot.div(combined_pivot.iloc[:,-1], axis=0 )
# combined_pivot.to_csv(Path("./data") / "combined_exec_pivot.csv")
# combined_pivot

## Tables for the thesis to give the complete overview

In [ ]:
combined_pivot_alternative = pd.pivot_table(combined_reset, values="target_filename", index=['dataset', "result_1"], columns=['model_name', 'template'], aggfunc=["sum"],
               fill_value=0,
               margins=True)
combined_pivot_alternative

## Table per language pair

Correct calculation of Percent


In [ ]:
import itertools

combined_pivot = combined_pivot[['success', 'compile', 'runtime','incorrect', 'All']]


combined_pivot_without_index = combined_pivot.reset_index()
combined_pivot_without_index

In [ ]:
# for each language pair:
languages = ["Python", "Java", "Go", "Rust", "C#"]

pl_permutations = itertools.permutations(languages, 2)
valid_permutations = set()
for pair in pl_permutations:
    if pair[0] in ["Python", "Java", "Go"]:  # "C", "C++":
        valid_permutations.add(pair)
for pair in valid_permutations:
    filtered = combined_pivot_without_index[(combined_pivot_without_index["source_lang"] == pair[0]) & (combined_pivot_without_index["target_lang"] == pair[1]) & (combined_pivot_without_index["template"] != "LIT")]
    for col_to_remove in ["source_lang", "target_lang"]:
        filtered = filtered.loc[:, filtered.columns != col_to_remove]
    iteration_1_stats = filtered.rename(columns={"model_name": "Model", "dataset": "Dataset", "template": "Prompt"}).groupby(['Dataset', 'Model', 'Prompt']).sum()
    s = iteration_1_stats.groupby(["Model", "Prompt"]).sum()
    index_frame = s.index.to_frame().reset_index(drop=True)
    index_frame["Dataset"] = "All"
    index_frame = index_frame.filter(['Dataset', 'Model', 'Prompt'])
    s.index = pd.MultiIndex.from_frame(index_frame)
    iteration_1_stats =pd.concat([iteration_1_stats, s]).sort_index().iloc[:, :]
    iteration_1_stats_percent = iteration_1_stats.div(iteration_1_stats.iloc[:,-1], axis=0) * 100
    iteration_1_stats_percent = iteration_1_stats_percent.stack().unstack(["Model", "Prompt"]).dropna(axis="columns")
    iteration_1_stats_percent.rename(inplace=True, columns={"codestral": "Codestral", "dolphin-2.6-mistral": "D-Mistral", "dolphin-2.6-phi-2": "D-Phi-2", "dolphin-2.7-mixtral": "D-Mixtral", "llama3-8b": "Llama 3", "phi3": "Phi-3", "mistral": "Mistral", "controlled_md": "MD", "via_description": "VT", "mixtral": "Mixtral", "controlled": "RM"}, index={"infinite loop": "loop"})
    iteration_1_stats.to_csv(f"./tables/iteration_1_stats{pair[0]}_{pair[1]}.csv")
    iteration_1_stats_percent.to_csv(f"./tables/iteration_1_stats_percent_{pair[0]}_{pair[1]}.csv")
    iteration_1_stats_percent.to_latex(f"./tables_tex/iteration_1_stats_percent_{pair[0]}_{pair[1]}.tex", float_format="%.2f")
print(pair)
iteration_1_stats_percent # show example


## All languages aggregated

In [ ]:
iteration_1_stats = combined_pivot.query("source_lang != 'Go'").groupby(['dataset', 'model_name', 'template']).sum()
s = iteration_1_stats.groupby(["model_name", "template"]).sum()
index_frame = s.index.to_frame().reset_index(drop=True)
index_frame["dataset"] = "All"
index_frame = index_frame.filter(['dataset', 'model_name', 'template'])
s.index = pd.MultiIndex.from_frame(index_frame)
iteration_1_stats =pd.concat([iteration_1_stats, s]).sort_index().iloc[1:, :]
iteration_1_stats.to_csv(f"./tables/iteration_1_stats_total.csv")
iteration_1_stats.index.rename({"model_name": "Model", "dataset": "Dataset", "template": "Prompt"}, inplace=True)
# iteration_1_stats

In [ ]:
iteration_1_stats_percent = iteration_1_stats.div(iteration_1_stats.iloc[:,-1], axis=0) * 100
iteration_1_stats_percent = iteration_1_stats_percent
#iteration_1_stats_percent

In [ ]:
iteration_1_stats_percent = iteration_1_stats_percent.stack().unstack(["Model", "Prompt"])
if opp == None:
    iteration_1_stats_percent = iteration_1_stats_percent.drop(columns=["LIT"], level=1)
iteration_1_stats_percent = iteration_1_stats_percent.dropna(axis="columns").round(2)
#iteration_1_stats_percent = iteration_1_stats_percent.stack().unstack(["Model", "Prompt"]).dropna(axis="columns").round(2)
iteration_1_stats_percent.rename(inplace=True, columns={"codestral": "Codestral", "dolphin-2.6-mistral": "D-Mistral", "dolphin-2.6-phi-2": "D-Phi-2", "dolphin-2.7-mixtral": "D-Mixtral", "llama3-8b": "Llama 3", "phi3": "Phi-3", "mistral": "Mistral", "controlled_md": "MD", "via_description": "VT", "mixtral": "Mixtral", "controlled": "RM"}, index={"infinite loop": "loop"})
if opp == None:
    iteration_1_stats_percent.to_csv(f"./tables/iteration_1_stats_percent_total.csv", float_format="%.2f")
    iteration_1_stats_percent.to_latex(f"./tables_tex/iteration_1_stats_percent_total.tex", float_format="%.2f")
else:
    iteration_1_stats_percent.to_csv(f"./tables/iteration_1_stats_percent_{opp}.csv", float_format="%.2f")
    iteration_1_stats_percent.to_latex(f"./tables_tex/iteration_1_stats_percent_{opp}.tex", float_format="%.2f")
iteration_1_stats_percent

Language analysis via table

In [ ]:
filtered = combined_pivot_without_index[combined_pivot_without_index["template"] == "controlled_md"]
for col_to_remove in ["template", 'dataset', 'model_name']:
    filtered = filtered.loc[:, filtered.columns != col_to_remove]
iteration_1_stats_controlled_md = filtered.groupby(['source_lang', 'target_lang']).sum()
iteration_1_stats_controlled_md_ratio = iteration_1_stats_controlled_md.div(iteration_1_stats_controlled_md.iloc[:,-1], axis=0) * 100

iteration_1_stats_controlled_md_ratio.iloc[:,:-1]#.plot(kind='bar')

## Plotting 

In [ ]:
from matplotlib.ticker import PercentFormatter
import numpy as np
def make_bar_plot_combined_pivot(filter_query: str, grouping: list, title: str | None = None, subgroup: str| None = None, percentage=True, only_success=True, xlabel=None, ylabel=None, bar_labels=False, xrot=0, label_col=5):
    all_index_col = ["dataset", "model_name", "template", "source_lang", "target_lang"]
    
    combined_pivot_without_index = combined_pivot.reset_index()
    filtered = combined_pivot_without_index.query(filter_query)
    for col_to_remove in [x for x in all_index_col if x not in grouping]:
        filtered = filtered.loc[:, filtered.columns != col_to_remove]
    graph_data = filtered.groupby(grouping).sum()
    if percentage:
        graph_data = graph_data.div(graph_data.iloc[:,-1], axis=0) * 100

    if subgroup and subgroup in grouping:
        index_level_subgroup = grouping.index(subgroup)
        graph_data=graph_data.unstack(index_level_subgroup)


    if only_success:
        graph_data=graph_data.loc[:, "success"]
    else:
        graph_data = graph_data.iloc[:,:-1]

    graph_data.rename(inplace=True, columns={"codestral": "Codestral", "dolphin-2.6-mistral": "D-Mistral", "dolphin-2.6-phi-2": "D-Phi-2", "dolphin-2.7-mixtral": "D-Mixtral", "llama3-8b": "Llama 3", "phi3": "Phi-3", "mistral": "Mistral", "controlled_md": "MD", "via_description": "VT", "mixtral": "Mixtral", "controlled": "RM"}, index={"infinite loop": "loop"})
    graph_data.rename(inplace=True, index={"codestral": "Codestral", "dolphin-2.6-mistral": "D-Mistral", "dolphin-2.6-phi-2": "D-Phi-2", "dolphin-2.7-mixtral": "D-Mixtral", "llama3-8b": "Llama 3", "phi3": "Phi-3", "mistral": "Mistral", "controlled_md": "MD", "via_description": "VT", "mixtral": "Mixtral", "controlled": "RM"}, columns={"infinite loop": "loop"})
    graph_data.rename(inplace=True, columns={"avatar": "AVATAR", "codenet": "CodeNet", "bithacks": "BitHacks"})
    graph_data.rename(inplace=True, index={"avatar": "AVATAR", "codenet": "CodeNet", "bithacks": "BitHacks"})
    print(graph_data)
    

    ax = graph_data.plot(kind='bar', title=title, xlabel=xlabel, ylabel=ylabel)
    ax.legend(bbox_to_anchor=(0, 1.02, 1, 0.2), loc="lower left",
                mode="expand", borderaxespad=0, ncol=label_col)
    ax.tick_params(axis='x', labelrotation=xrot)
    if percentage:
        ax.set_ylim([0, 100])
        ax.yaxis.set_major_formatter(PercentFormatter())
    if bar_labels:
        for container in ax.containers:
            ax.bar_label(container, fmt='%.2f')



In [ ]:

make_bar_plot_combined_pivot(
    "template == 'controlled_md'",
    ['dataset', 'model_name'],
    percentage=True,
    only_success=True,
    xlabel="Dataset",
    ylabel="Percentage of Successful Samples",
    xrot=90,
    subgroup="model_name"
)

make_bar_plot_combined_pivot(
    "template == 'controlled_md'",
    ['dataset', 'model_name'],
    percentage=True,
    only_success=True,
    xlabel="Model",
    ylabel="Percentage of Successful Samples",
    xrot=90,
    subgroup="dataset"
)

In [ ]:
# Variance in the outcomes by model for the controlled_md template
# Classification of the outputs of the initial translation round \nwith the controlled_md template
make_bar_plot_combined_pivot(
    "template == 'controlled_md'",
    ['model_name'],
    percentage=True,
    only_success=False,
    xlabel="Model",
    ylabel="Outcome Rate",
    xrot=-45
)
#### TODO links nach rechts unten ticks anschrägen

In [ ]:
# Variance in the outcomes by template for mistral-7b
make_bar_plot_combined_pivot(
    "model_name == 'mistral'",
    ['template'],
    percentage=True,
    only_success=False,
    xlabel="Prompt Template",
    ylabel="Percentage of Samples"
)
# Variance in the outcomes by template for dolphin-2.6-mistral-7b
make_bar_plot_combined_pivot(
    "model_name == 'dolphin-2.6-mistral'",
    ['template'],
    percentage=True,
    only_success=False,
    xlabel="Prompt Template",
    ylabel="Percentage of Samples"
)



In [ ]:
# TODO create subplots by Source PL
# make_bar_plot_combined_pivot(
#     "template == 'controlled_md'",
#     ['source_lang', 'target_lang'],
#     percentage=True,
#     only_success=False,
#     xlabel="Language Pair",
#     ylabel="Percentage of Samples",
#     xrot=-45
# )


# subplots by Source PL
make_bar_plot_combined_pivot(
    "template == 'controlled_md'",
    ['source_lang', 'target_lang'],
    percentage=True,
    only_success=True,
    xlabel="Source PL",
    ylabel="Success Rate",
    xrot=-45,
    subgroup="target_lang"
)
print("llama 3")
make_bar_plot_combined_pivot(
    "template == 'controlled_md' and model_name == 'llama3-8b'",
    ['source_lang', 'target_lang'],
    percentage=True,
    only_success=True,
    xlabel="Source PL",
    ylabel="Success Rate",
    xrot=-45,
    subgroup="target_lang"
)
print("D-Phi-2")
make_bar_plot_combined_pivot(
    "template == 'controlled_md' and model_name == 'dolphin-2.6-phi-2'",
    ['source_lang', 'target_lang'],
    percentage=True,
    only_success=True,
    xlabel="Source PL",
    ylabel="Success Rate",
    xrot=-45,
    subgroup="target_lang"
)
make_bar_plot_combined_pivot(
    "template == 'controlled' and model_name == 'dolphin-2.6-phi-2'",
    ['source_lang', 'target_lang'],
    percentage=True,
    only_success=True,
    xlabel="Source PL",
    ylabel="Success Rate",
    xrot=-45,
    subgroup="target_lang"
)
print("phi3")
make_bar_plot_combined_pivot(
    "template == 'controlled_md' and model_name == 'phi3'",
    ['source_lang', 'target_lang'],
    percentage=True,
    only_success=True,
    xlabel="Source PL",
    ylabel="Success Rate",
    xrot=-45,
    subgroup="target_lang"
)
print("codestral")
make_bar_plot_combined_pivot(
    "template == 'controlled_md' and model_name == 'codestral'",
    ['source_lang', 'target_lang'],
    percentage=True,
    only_success=True,
    xlabel="Source PL",
    ylabel="Success Rate",
    xrot=0,
    subgroup="target_lang"
)
plt.savefig("./plots/codestral_lanugage_pairs.pdf")
print("D-Mixtral")
make_bar_plot_combined_pivot(
    "template == 'controlled_md' and model_name == 'dolphin-2.7-mixtral'",
    ['source_lang', 'target_lang'],
    percentage=True,
    only_success=True,
    xlabel="Source PL",
    ylabel="Success Rate",
    xrot=-45,
    subgroup="target_lang"
)
print("Mixtral")
make_bar_plot_combined_pivot(
    "template == 'controlled_md' and model_name == 'mixtral'",
    ['source_lang', 'target_lang'],
    percentage=True,
    only_success=True,
    xlabel="Source PL",
    ylabel="Success Rate",
    xrot=-45,
    subgroup="target_lang"
)
print("D-Mistral")
make_bar_plot_combined_pivot(
    "template == 'controlled_md' and model_name == 'dolphin-2.6-mistral'",
    ['source_lang', 'target_lang'],
    percentage=True,
    only_success=True,
    xlabel="Source PL",
    ylabel="Success Rate",
    xrot=-45,
    subgroup="target_lang"
)
print("Mistral")
make_bar_plot_combined_pivot(
    "template == 'controlled_md' and model_name == 'mistral'",
    ['source_lang', 'target_lang'],
    percentage=True,
    only_success=True,
    xlabel="Source PL",
    ylabel="Success Rate",
    xrot=-45,
    subgroup="target_lang"
)


In [ ]:

# Variance in the outcomes by source PL for Codestral
# make_bar_plot_combined_pivot(
#     "model_name == 'codestral'",
#     ['source_lang', 'target_lang'],
#     percentage=True,
#     only_success=False,
#     xlabel="Language Pair",
#     ylabel="Outcome Rate",
#     xrot=-90
# )


In [ ]:
# Variance in the outcomes by source PL for Codestral
make_bar_plot_combined_pivot(
   "template == 'controlled_md'",# "model_name.str.contains('mixtral', na=False) or model_name.str.contains('mistral', na=False)",
    ['target_lang', 'model_name'],
    percentage=True,
    only_success=True,
    xlabel="Target Language",
    ylabel="Success Rate",
    xrot=45,
    subgroup="model_name"
)

# Variance in the outcomes by source PL for Codestral
make_bar_plot_combined_pivot(
    "model_name.str.contains('mixtral', na=False)",
    ['target_lang', 'model_name'],
    percentage=True,
    only_success=True,
    xlabel="Target Language",
    ylabel="Success Rate",
    xrot=45,
    subgroup="model_name"
)

In [ ]:
# Variance in the outcomes for Codestral
make_bar_plot_combined_pivot(
    "model_name == 'codestral'",
    ['template'],
    percentage=True,
    only_success=False,
    xlabel="Outcomes for translations",
    ylabel="Outcome Rate",
    bar_labels=True
)

In [ ]:
# Variance in the outcomes by source PL for Codestral
make_bar_plot_combined_pivot(
    "model_name == 'codestral'",
    ['source_lang'],
    percentage=True,
    only_success=False,
    xlabel="Source PL",
    ylabel="Outcome Rate"
)

use the next diagram

In [ ]:
# "Variance in the Outcomes by Target PL for Codestral"
make_bar_plot_combined_pivot(
    "model_name == 'codestral'",
    ['target_lang'],
    percentage=True,
    only_success=False,
    xlabel="Target PL",
    ylabel="Outcome Rate",
    xrot=-45
)

make_bar_plot_combined_pivot(
    "model_name == 'dolphin-2.7-mixtral'",
    ['target_lang'],
    percentage=True,
    only_success=False,
    xlabel="Target PL",
    ylabel="Outcome Rate",
    xrot=-45
)

# Results iterative approach

In [ ]:
combined_without_faluty_source = raw_combined.query("result_1 != 'faulty source' and result_1 != 'unicode error'")#.query("target_lang == 'Go'")
combined_iterative = combined_without_faluty_source[['dataset', 'source_lang', 'target_lang', 'model_name', 'template', 'result_1', 'result_2', 'result_3', 'target_filename']]#.groupby(['dataset', 'model_name', 'template', 'source_lang', 'target_lang', 'result_1']).count()
combined_iterative

In [ ]:
model_template_iteration_2 = list(combined_iterative[['model_name', 'template', 'result_2', 'target_filename']].groupby(['model_name', 'template', 'result_2']).count().reset_index()[["model_name", "template"]].value_counts().index)
model_template_iteration_2

In [ ]:
current_df = combined_without_faluty_source[['dataset', 'source_lang', 'target_lang', 'model_name', 'template', 'result_1', 'result_2', 'target_filename']]
# Group the DataFrame by 'model_name' and 'template', then count the number of non-NaN results for each group
grouped_df = current_df.groupby(['model_name', 'template']).result_2.count().reset_index()
grouped_df.columns = ['model_name', 'template', 'count_non_nan_result_2']
# Filter the original DataFrame to only include rows where the count of non-NaN results is greater than 0 (there was a 2nd iteration)
df_filtered = current_df.merge(grouped_df, on=['model_name', 'template']).drop_duplicates(subset=['dataset', 'source_lang', 'target_lang', 'model_name', 'template', 'result_1', 'result_2', 'target_filename'])
df_filtered = df_filtered[df_filtered['count_non_nan_result_2'] > 0]
df_filtered

In [ ]:
df_filtered.drop_duplicates(["model_name", "template"])[["model_name", "template", "count_non_nan_result_2"]].reset_index()


In [ ]:
results_total_second_iteration = df_filtered[['dataset', 'source_lang', 'target_lang', 'model_name', 'template', 'result_1', 'result_2', 'target_filename']].fillna({"result_2": "success"})
results_total_second_iteration.groupby(['dataset', 'model_name', 'template', 'source_lang', 'target_lang'])

grouped = results_total_second_iteration.groupby(['dataset', 'model_name', 'template', 'source_lang', 'target_lang'])
result_1_count = grouped['result_1'].apply(lambda x: pd.Series(x).value_counts())
result_2_count = grouped['result_2'].apply(lambda x: pd.Series(x).value_counts())
combined_count = pd.merge(result_1_count.to_frame(), result_2_count.to_frame(), how="outer", left_index=True, right_index=True).fillna(0).convert_dtypes("int")
combined_count["change 1->2"] = combined_count["result_2"] - combined_count['result_1']
combined_count

All language pairs combined

In [ ]:
template_fix = "controlled_md"


In [ ]:
grouped = results_total_second_iteration.groupby(['dataset', 'model_name', 'template'])
result_1_count = grouped['result_1'].apply(lambda x: pd.Series(x).value_counts())
result_2_count = grouped['result_2'].apply(lambda x: pd.Series(x).value_counts())
combined_count = pd.merge(result_1_count.to_frame(), result_2_count.to_frame(), how="outer", left_index=True, right_index=True).fillna(0).convert_dtypes("int")
combined_count["change 1->2"] = combined_count["result_2"] - combined_count['result_1']
s = combined_count.groupby(['dataset', 'model_name', 'template']).sum()
s["change 1->2"] = s["result_1"]
#s = iteration_1_stats.groupby(["model_name", "template"]).sum()
index_frame = s.index.to_frame().reset_index(drop=True)
index_frame["Category"] = "Total"
index_frame = index_frame.filter(['dataset', 'model_name', 'template', 'Category'])
print(index_frame)
s.index = pd.MultiIndex.from_frame(index_frame)
combined_count =pd.concat([combined_count, s]).sort_index().iloc[:, :]

combined_count

all_ds = combined_count.unstack(-1).groupby(["model_name", "template"]).sum()
index_frame = all_ds.index.to_frame().reset_index(drop=True)
index_frame["dataset"] = "All"
index_frame = index_frame.filter(['dataset', 'model_name', 'template'])
all_ds.index = pd.MultiIndex.from_frame(index_frame)
combined_count_all =pd.concat([combined_count.unstack(-1), all_ds]).sort_index().iloc[:, :]
combined_count_all = combined_count_all.stack()


In [ ]:
grouped = results_total_second_iteration.groupby(['dataset', 'model_name', 'template'])
result_1_count = grouped['result_1'].apply(lambda x: pd.Series(x).value_counts())
result_2_count = grouped['result_2'].apply(lambda x: pd.Series(x).value_counts())
combined_count = pd.merge(result_1_count.to_frame(), result_2_count.to_frame(), how="outer", left_index=True, right_index=True).fillna(0).convert_dtypes("int")
combined_count["change 1->2"] = combined_count["result_2"] - combined_count['result_1']
s = combined_count.groupby(['dataset', 'model_name', 'template']).sum()
s["change 1->2"] = s["result_1"]
#s = iteration_1_stats.groupby(["model_name", "template"]).sum()
index_frame = s.index.to_frame().reset_index(drop=True)
index_frame["Category"] = "Total"
index_frame = index_frame.filter(['dataset', 'model_name', 'template', 'Category'])
print(index_frame)
s.index = pd.MultiIndex.from_frame(index_frame)
combined_count =pd.concat([combined_count, s]).sort_index().iloc[:, :]

combined_count

all_ds = combined_count.unstack(-1).groupby(["model_name", "template"]).sum()
index_frame = all_ds.index.to_frame().reset_index(drop=True)
index_frame["dataset"] = "All"
index_frame = index_frame.filter(['dataset', 'model_name', 'template'])
all_ds.index = pd.MultiIndex.from_frame(index_frame)
combined_count_all =pd.concat([combined_count.unstack(-1), all_ds]).sort_index().iloc[:, :]
combined_count_all = combined_count_all.stack()


In [ ]:
if template_fix == "controlled_md":
    grouped = results_total_second_iteration.query("template == 'controlled_md'").drop(columns="template").groupby(['dataset', 'model_name'])
    result_1_count = grouped['result_1'].apply(lambda x: pd.Series(x).value_counts())
    result_2_count = grouped['result_2'].apply(lambda x: pd.Series(x).value_counts())
    combined_count = pd.merge(result_1_count.to_frame(), result_2_count.to_frame(), how="outer", left_index=True, right_index=True).fillna(0).convert_dtypes("int")
    combined_count["Diff."] = combined_count["result_2"] - combined_count['result_1']
    s = combined_count.groupby(['dataset', 'model_name']).sum()
    s["Diff."] = s["result_1"]
    #s = iteration_1_stats.groupby(["model_name", "template"]).sum()
    index_frame = s.index.to_frame().reset_index(drop=True)
    index_frame["Category"] = "Total"
    index_frame = index_frame.filter(['dataset', 'model_name', 'Category'])
    print(index_frame)
    s.index = pd.MultiIndex.from_frame(index_frame)
    combined_count = pd.concat([combined_count, s]).sort_index().iloc[:, :]

    combined_count.drop(columns=["result_1"], inplace=True)
    combined_count.rename(columns={"result_2": "Repaired"}, inplace=True)

    all_ds = combined_count.unstack(-1).groupby(["model_name"]).sum()
    index_frame = all_ds.index.to_frame().reset_index(drop=True)
    index_frame["dataset"] = "All"
    index_frame = index_frame.filter(['dataset', 'model_name'])
    all_ds.index = pd.MultiIndex.from_frame(index_frame)
    combined_count_all =pd.concat([combined_count.unstack(-1), all_ds]).sort_index().iloc[:, :]
    combined_count_all = combined_count_all.stack()

    tmp = combined_count_all.unstack(-1).stack(0)
    tmp = tmp.div(tmp["Total"], axis=0) * 100
    combined_count_percent = tmp.unstack(-1).stack(0)
    combined_count_percent = combined_count_percent.stack().unstack(["model_name"]).fillna(0).unstack(-1)
    combined_count_percent.rename(inplace=True, columns={"codestral": "Codestral", "dolphin-2.6-mistral": "D-Mistral", "dolphin-2.6-phi-2": "D-Phi-2", "dolphin-2.7-mixtral": "D-Mixtral", "llama3-8b": "Llama 3", "phi3": "Phi-3", "mistral": "Mistral", "controlled_md": "MD", "via_description": "VT", "mixtral": "Mixtral", "controlled": "RM"}, index={"infinite loop": "loop"})

    combined_count_percent.to_csv("./tables/iteration_2_percent.csv")
    combined_count_percent.to_excel("./tables/iteration_2_percent.xlsx", float_format="%.2f")
    combined_count_percent.to_latex("./tables_tex/iteration_2_percent.tex", float_format="%.2f")
combined_count_percent

#combined_count

In [ ]:
combined_count_numbers = combined_count_all.stack().unstack(["model_name", "template"]).fillna(0).unstack(-1)
combined_count_numbers.rename(inplace=True, columns={"codestral": "Codestral", "dolphin-2.6-mistral": "D-Mistral", "dolphin-2.6-phi-2": "D-Phi-2", "dolphin-2.7-mixtral": "D-Mixtral", "llama3-8b": "Llama 3", "phi3": "Phi-3", "mistral": "Mistral", "controlled_md": "MD", "via_description": "VT", "mixtral": "Mixtral", "controlled": "RM"}, index={"infinite loop": "loop"})
combined_count_numbers.to_csv("./tables/iteration_2_numbers.csv", float_format="")
combined_count_numbers

In [ ]:
tmp = combined_count_all.unstack(-1).stack(0)
tmp = tmp.div(tmp["Total"], axis=0) * 100
combined_count_percent = tmp.unstack(-1).stack(0)
combined_count_percent = combined_count_percent.stack().unstack(["model_name", "template"]).fillna(0).unstack(-1)
combined_count_percent.rename(inplace=True, columns={"codestral": "Codestral", "dolphin-2.6-mistral": "D-Mistral", "dolphin-2.6-phi-2": "D-Phi-2", "dolphin-2.7-mixtral": "D-Mixtral", "llama3-8b": "Llama 3", "phi3": "Phi-3", "mistral": "Mistral", "controlled_md": "MD", "via_description": "VT", "mixtral": "Mixtral", "controlled": "RM"}, index={"infinite loop": "loop"})

combined_count_percent.to_csv("./tables/iteration_2_percent.csv")
combined_count_percent.to_excel("./tables/iteration_2_percent.xlsx", float_format="%.2f")
combined_count_percent.to_latex("./tables_tex/iteration_2_percent.tex", float_format="%.2f")
combined_count_percent

Iteration 3

In [ ]:
current_df = combined_without_faluty_source[['dataset', 'source_lang', 'target_lang', 'model_name', 'template', 'result_1', 'result_2', 'result_3', 'target_filename']]
# Group the DataFrame by 'model_name' and 'template', then count the number of non-NaN results for each group
grouped_df = current_df.groupby(['model_name', 'template']).result_3.count().reset_index()
grouped_df.columns = ['model_name', 'template', 'count_non_nan_result_3']
# Filter the original DataFrame to only include rows where the count of non-NaN results is greater than 0 (there was a 3nd iteration)
df_filtered = current_df.merge(grouped_df, on=['model_name', 'template']).drop_duplicates(subset=['dataset', 'source_lang', 'target_lang', 'model_name', 'template', 'result_1', 'result_2', 'result_3', 'target_filename'])
df_filtered = df_filtered[df_filtered['count_non_nan_result_3'] > 0]
df_filtered
results_total_third_iteration = df_filtered[['dataset', 'source_lang', 'target_lang', 'model_name', 'template', 'result_1', 'result_2', 'result_3', 'target_filename']].fillna({"result_2": "success", "result_3": "success"})
results_total_third_iteration.groupby(['dataset', 'model_name', 'template', 'source_lang', 'target_lang'])

grouped = results_total_third_iteration.groupby(['dataset', 'model_name', 'template']) #, 'source_lang', 'target_lang'])
result_1_count = grouped['result_1'].apply(lambda x: pd.Series(x).value_counts())
result_2_count = grouped['result_2'].apply(lambda x: pd.Series(x).value_counts())
result_3_count = grouped['result_3'].apply(lambda x: pd.Series(x).value_counts())
combined_count = pd.merge(result_1_count.to_frame(), result_2_count.to_frame(), how="outer", left_index=True, right_index=True).fillna(0).convert_dtypes("int")
combined_count = pd.merge(combined_count, result_3_count.to_frame(), how="outer", left_index=True, right_index=True).fillna(0).convert_dtypes("int")
combined_count["change 1->2"] = combined_count["result_2"] - combined_count['result_1']
combined_count["change 2->3"] = combined_count["result_3"] - combined_count['result_2']
combined_count["change 1->3"] = combined_count["result_3"] - combined_count['result_1']
combined_count


In [ ]:
s = combined_count.groupby(['dataset', 'model_name', 'template']).sum()
s["change 1->2"] = s["result_1"]
s["change 2->3"] = s["result_1"]
s["change 1->3"] = s["result_1"]

index_frame = s.index.to_frame().reset_index(drop=True)
index_frame["Category"] = "Total"
index_frame = index_frame.filter(['dataset', 'model_name', 'template', 'Category'])
print(index_frame)
s.index = pd.MultiIndex.from_frame(index_frame)
combined_count_with_total =pd.concat([combined_count, s]).sort_index()
combined_count_with_total

In [ ]:

grouped = results_total_third_iteration.groupby(['dataset', 'model_name', 'template', 'source_lang', 'target_lang'])
result_1_count = grouped['result_1'].apply(lambda x: pd.Series(x).value_counts())
result_2_count = grouped['result_2'].apply(lambda x: pd.Series(x).value_counts())
result_3_count = grouped['result_3'].apply(lambda x: pd.Series(x).value_counts())
combined_count = pd.merge(result_1_count.to_frame(), result_2_count.to_frame(), how="outer", left_index=True, right_index=True).fillna(0).convert_dtypes("int")
combined_count["change 1->2"] = combined_count["result_2"] - combined_count['result_1']
combined_count["change 2->3"] = combined_count["result_3"] - combined_count['result_2']
combined_count["change 1->3"] = combined_count["result_3"] - combined_count['result_1']
combined_count

In [ ]:
tmp = combined_count_with_total.unstack(-1).stack(0)
tmp = tmp.div(tmp["Total"], axis=0) * 100
combined_count_percent = tmp.unstack(-1).stack(0)
combined_count_percent = combined_count_percent.stack().unstack(["model_name", "template"]).fillna(0).unstack(-1)
combined_count_percent

In [ ]:
combined_count_with_total.unstack().groupby(["model_name", "template"]).sum().stack()

In [ ]:
tmp = combined_count_with_total.unstack().groupby(["model_name", "template"]).sum().stack().unstack(-1).stack(0)
tmp = tmp.div(tmp["Total"], axis=0) * 100
combined_count_percent = tmp.unstack(-1).stack(0)
combined_count_percent = combined_count_percent.stack().unstack(["model_name", "template"]).fillna(0).unstack(-1)
combined_count_percent

Try Sankey


In [ ]:
current_df = combined_without_faluty_source[['dataset', 'source_lang', 'target_lang', 'model_name', 'template', 'result_1', 'result_2', 'result_3', 'target_filename']]
# Group the DataFrame by 'model_name' and 'template', then count the number of non-NaN results for each group
grouped_df = current_df.groupby(['model_name', 'template']).result_3.count().reset_index()
grouped_df.columns = ['model_name', 'template', 'count_non_nan_result_3']
# Filter the original DataFrame to only include rows where the count of non-NaN results is greater than 0 (there was a 3nd iteration)
df_filtered = current_df.merge(grouped_df, on=['model_name', 'template']).drop_duplicates(subset=['dataset', 'source_lang', 'target_lang', 'model_name', 'template', 'result_1', 'result_2', 'result_3', 'target_filename'])
df_filtered = df_filtered[df_filtered['count_non_nan_result_3'] > 0]
df_filtered
results_total_third_iteration = df_filtered[['dataset', 'source_lang', 'target_lang', 'model_name', 'template', 'result_1', 'result_2', 'result_3', 'target_filename']].fillna({"result_2": "success", "result_3": "success"})
results_total_third_iteration